In [2]:
#Importing libraries
import sys
import json
import pymongo
import os

In [3]:
#Connecting Mongo
client = pymongo.MongoClient("localhost", 27017)

In [4]:
#Looking database names
client.database_names()

C:\Users\Ivan_\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: database_names is deprecated. Use list_database_names instead.
  


['admin',
 'config',
 'datos',
 'local',
 'steam_data',
 'steam_datos',
 'test',
 'twitter']

In [5]:
#Selecting database
db_twittes = client.twitter

In [6]:
#Looking collections names
db_twittes.collection_names()

C:\Users\Ivan_\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: collection_names is deprecated. Use list_collection_names instead.
  


['raw_tweets', 'clean_tweets']

In [7]:
#Selecting collection
raw = db_twittes.raw_tweets

In [8]:
#Verifying one document
raw.find_one()

{'_id': ObjectId('5c5daf1a9d98244eefaee286'),
 'contributors': None,
 'coordinates': None,
 'created_at': 'Sun Jan 06 20:08:07 +0000 2019',
 'entities': {'hashtags': [], 'symbols': [], 'urls': [], 'user_mentions': []},
 'favorite_count': 0,
 'favorited': False,
 'filter_level': 'low',
 'geo': None,
 'id': 1082005915698716672,
 'id_str': '1082005915698716672',
 'in_reply_to_screen_name': None,
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'is_quote_status': False,
 'lang': 'pt',
 'place': None,
 'quote_count': 0,
 'reply_count': 0,
 'retweet_count': 0,
 'retweeted': False,
 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>',
 'text': 'João Paulo é muito chato kkkkkkkkkkk',
 'timestamp_ms': '1546805287657',
 'truncated': False,
 'user': {'contributors_enabled': False,
  'created_at': 'Tue Jul 31 15:30:26 +0000 2018',
  'default_profile': True,
  'default_pr

In [9]:
#Counting documents in English
raw.find({"lang":"en"}).count()

C:\Users\Ivan_\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  


2893

In [10]:
#Creating index of type Text
raw.create_index([("lang",pymongo.TEXT)], name="search_index", default_language="english")

'search_index'

In [11]:
#Index Information
raw.index_information()

{'_id_': {'key': [('_id', 1)], 'ns': 'twitter.raw_tweets', 'v': 2},
 'lang_1': {'key': [('lang', 1)], 'ns': 'twitter.raw_tweets', 'v': 2},
 'search_index': {'default_language': 'english',
  'key': [('_fts', 'text'), ('_ftsx', 1)],
  'language_override': 'language',
  'ns': 'twitter.raw_tweets',
  'textIndexVersion': 3,
  'v': 2,
  'weights': SON([('lang', 1)])},
 'search_index_1': {'key': [('search_index', 1)],
  'ns': 'twitter.raw_tweets',
  'v': 2}}

In [12]:
#Veritying if the seach is using the index correctly
raw.find({"lang":"en"}).explain()

{'executionStats': {'allPlansExecution': [],
  'executionStages': {'advanced': 2893,
   'alreadyHasObj': 0,
   'docsExamined': 2893,
   'executionTimeMillisEstimate': 33,
   'inputStage': {'advanced': 2893,
    'direction': 'forward',
    'dupsDropped': 0,
    'dupsTested': 0,
    'executionTimeMillisEstimate': 0,
    'indexBounds': {'lang': ['["en", "en"]']},
    'indexName': 'lang_1',
    'indexVersion': 2,
    'invalidates': 0,
    'isEOF': 1,
    'isMultiKey': False,
    'isPartial': False,
    'isSparse': False,
    'isUnique': False,
    'keyPattern': {'lang': 1},
    'keysExamined': 2893,
    'multiKeyPaths': {'lang': []},
    'nReturned': 2893,
    'needTime': 0,
    'needYield': 0,
    'restoreState': 23,
    'saveState': 23,
    'seeks': 1,
    'seenInvalidated': 0,
    'stage': 'IXSCAN',
    'works': 2894},
   'invalidates': 0,
   'isEOF': 1,
   'nReturned': 2893,
   'needTime': 0,
   'needYield': 0,
   'restoreState': 23,
   'saveState': 23,
   'stage': 'FETCH',
   'works':

In [54]:
#Filtering documents with the field "Text" bigger than 16 characters
new = raw.find({"lang":"en","text":{"$exists":True},"$where": "this.text.length>16"})

#### OBS: Some documents there are more than one field called "Text", when this happens if one of those fields matchs the requirement, the document is selected. We will see one problem that happens later on.

In [55]:
#Creating new collection
clean = db_twittes["clean_tweets"]

In [56]:
#Inserting documents into the new collection
clean.insert(new)

C:\Users\Ivan_\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  """Entry point for launching an IPython kernel.


[ObjectId('5c5daf1a9d98244eefaee288'),
 ObjectId('5c5daf1a9d98244eefaee28a'),
 ObjectId('5c5daf1a9d98244eefaee28b'),
 ObjectId('5c5daf1a9d98244eefaee28e'),
 ObjectId('5c5daf1a9d98244eefaee28f'),
 ObjectId('5c5daf1a9d98244eefaee292'),
 ObjectId('5c5daf1a9d98244eefaee293'),
 ObjectId('5c5daf1a9d98244eefaee295'),
 ObjectId('5c5daf1a9d98244eefaee299'),
 ObjectId('5c5daf1a9d98244eefaee2a6'),
 ObjectId('5c5daf1a9d98244eefaee2a8'),
 ObjectId('5c5daf1a9d98244eefaee2af'),
 ObjectId('5c5daf1a9d98244eefaee2b4'),
 ObjectId('5c5daf1a9d98244eefaee2bd'),
 ObjectId('5c5daf1a9d98244eefaee2c1'),
 ObjectId('5c5daf1a9d98244eefaee2c2'),
 ObjectId('5c5daf1a9d98244eefaee2c4'),
 ObjectId('5c5daf1a9d98244eefaee2d1'),
 ObjectId('5c5daf1a9d98244eefaee2d2'),
 ObjectId('5c5daf1a9d98244eefaee2d3'),
 ObjectId('5c5daf1a9d98244eefaee2d5'),
 ObjectId('5c5daf1a9d98244eefaee2d6'),
 ObjectId('5c5daf1a9d98244eefaee2da'),
 ObjectId('5c5daf1a9d98244eefaee2dc'),
 ObjectId('5c5daf1a9d98244eefaee2de'),
 ObjectId('5c5daf1a9d9824

In [153]:
#Verifying if the documents were well inserted
clean.find_one()

{'_id': ObjectId('5c5daf1a9d98244eefaee288'),
 'contributors': None,
 'coordinates': None,
 'created_at': 'Sun Jan 06 20:08:07 +0000 2019',
 'display_text_range': [12, 66],
 'entities': {'hashtags': [],
  'symbols': [],
  'urls': [],
  'user_mentions': [{'id': 1561123663,
    'id_str': '1561123663',
    'indices': [0, 11],
    'name': 'B/R Football',
    'screen_name': 'brfootball'}]},
 'favorite_count': 0,
 'favorited': False,
 'filter_level': 'low',
 'geo': None,
 'id': 1082005915723804672,
 'id_str': '1082005915723804672',
 'in_reply_to_screen_name': 'brfootball',
 'in_reply_to_status_id': 1082005301514199040,
 'in_reply_to_status_id_str': '1082005301514199040',
 'in_reply_to_user_id': 1561123663,
 'in_reply_to_user_id_str': '1561123663',
 'is_quote_status': False,
 'lang': 'en',
 'place': None,
 'quote_count': 0,
 'reply_count': 0,
 'retweet_count': 0,
 'retweeted': False,
 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>',
 'text': "@

In [48]:
#Exporting collection as an archive of type .txt
os.system("mongoexport -d twitter -c clean_tweets -o C:/Users/Ivan_/_Jupyter/Mongo/clean_tweets.txt")

0

In [138]:
#Extracting the scores from the tweets

def lines(fp):
    print(str(len(fp.readlines())))
    
def main():
    sent_file = "C:/Users/Ivan_/_Jupyter/Mongo/AFINN-111.txt"
    tweet_file = open("C:/Users/Ivan_/_Jupyter/Mongo/clean_tweets.txt",encoding="utf8")
    scores = term_scores(sent_file)
    for tweet in tweet_file:
    	#print tweet
    	print(tweet_score(tweet, scores))
        
def term_scores(score_file):
    afinnfile = open(score_file)
    scores = {} # initialize an empty dictionary
    for line in afinnfile:
        term, score  = line.split("\t")  # The file is tab-delimited. "\t" means "tab character"
        scores[term] = int(score)  # Convert the score to an integer.
    #print scores.items() # Print every (term, score) pair in the dictionary
    return scores

def tweet_score(text, scores):
    tweet = json.loads(text)
    score = 0

    if ('text' in tweet):
        terms = tweet['text'].lower().split(" ")
        print (tweet['text'])

    for term in terms:
        if (term in scores):
            score += scores[term]
    
    return clean.update_one( { "text" : tweet['text'] },{ "$set": { "score" : score } } ), score

if __name__ == '__main__':
    main()


@brfootball That means we'll expect him to ghost in the CL QFs💀...
(<pymongo.results.UpdateResult object at 0x000001BE0A488E88>, -1)
@apollozac Ahhh, gotcha. So it doesn’t effect performance?
(<pymongo.results.UpdateResult object at 0x000001BE0A4884C8>, 0)
@Omnia42022457 Send DM
(<pymongo.results.UpdateResult object at 0x000001BE0A488588>, 0)
RT @ABC7Erin: #BreakingNews: DC United’s Wayne Rooney was arrested on public intoxication and swearing charges, sources tell @abc7news #MLS…
(<pymongo.results.UpdateResult object at 0x000001BE0A48DA88>, -5)
And we go to the penalty kill with 1:00 left in the third. Score is 6-3 Rivs #CTWvsMET
(<pymongo.results.UpdateResult object at 0x000001BE0A488808>, -5)
RT @ctrinity9: the friends I got rn... we family fr
(<pymongo.results.UpdateResult object at 0x000001BE0A488748>, 0)
🤣🤣🤣🤣🤣 @MCATEER4  didnt kick him half hard enough
(<pymongo.results.UpdateResult object at 0x000001BE0A48D248>, -1)
WE DID IT! 

50 RT in less than 24 hours! You guys are the best

(<pymongo.results.UpdateResult object at 0x000001BE0A48D248>, 0)
RT @iamsteveolaa: Men go shopping to buy what they want; And women go shopping to find out what they want! Wonderful Creatures 😂😂😂
(<pymongo.results.UpdateResult object at 0x000001BE0A488C88>, 4)
I'm looking over my shoulder for a fuck to give Since yall convinced there's probably more left
(<pymongo.results.UpdateResult object at 0x000001BE0A488B08>, -3)
That moment when u lit up an idea then suddenly running short on fuse. I think it's not just the learning, is it? I… https://t.co/oO3HU38FuV
(<pymongo.results.UpdateResult object at 0x000001BE0A488308>, 0)
pam knows a lot of things abt the egg fic for somebody who hasn’t read it
(<pymongo.results.UpdateResult object at 0x000001BE0A488048>, 0)
😍 — watching Love 020 - Chinese Drama
(<pymongo.results.UpdateResult object at 0x000001BE0A488608>, 3)
Women are the forgotten people of the modern Liberal Party. https://t.co/cQjVJG0JR4 via @AusIndiMedia
(<pymongo.results.UpdateRes

(<pymongo.results.UpdateResult object at 0x000001BE0A488908>, 6)
my family is the best! https://t.co/ZZKbqGZy9Q
(<pymongo.results.UpdateResult object at 0x000001BE0A495D08>, 0)
RT @SethAbramson: Kindergarten common sense is now a "profile in courage" inside the GOP 🤦‍♂️ https://t.co/h1BxeCve4S
(<pymongo.results.UpdateResult object at 0x000001BE0A488688>, 0)
RT @kabiru_shema: The most beautiful thing I’ve read so far today. May you be very rich and happy. https://t.co/jfLYVznSsh
(<pymongo.results.UpdateResult object at 0x000001BE0A488848>, 5)
RT @CJTooSmooth: love me a girl with glasses. bring that blind ass here baby. no…over here smh
(<pymongo.results.UpdateResult object at 0x000001BE0A495848>, -2)
@rxurs you still leading lucid?
(<pymongo.results.UpdateResult object at 0x000001BE0A495948>, 0)
RT @sankrant: .@narendramodi &amp; @PrakashJavdekar should celebrate massive closure of Hindu-run schools. #RTE working as intended. 
https://t…
(<pymongo.results.UpdateResult object at 0x000001

(<pymongo.results.UpdateResult object at 0x000001BE0DF835C8>, 0)
@MyFavsTrash @Twitter read this shit. He woke af
(<pymongo.results.UpdateResult object at 0x000001BE0A495708>, 0)
RT @eleven8: The second video where she’s talking about it behind a very rough and tumultuous period. I’ve never seen it before and that ma…
(<pymongo.results.UpdateResult object at 0x000001BE0A488308>, 0)
Kites on the mall today https://t.co/YCRLMRWFgU
(<pymongo.results.UpdateResult object at 0x000001BE0A488208>, 0)
RT @KLewis_717: Tampico is the thickest juice in the world you can damn near chew it https://t.co/cq2Qg3TDSf
(<pymongo.results.UpdateResult object at 0x000001BE0A4889C8>, -4)
RT @muvaangel: Honestly this the best white girl twerk I’ve ever seen lmao. https://t.co/rTYrp5PH7x
(<pymongo.results.UpdateResult object at 0x000001BE0A488DC8>, 3)
Feels like it’s close..
(<pymongo.results.UpdateResult object at 0x000001BE0A48DF88>, 2)
We have an imbecile petty president who is to dumb too realize that this 

(<pymongo.results.UpdateResult object at 0x000001BE0A495948>, 2)
RT @ohgladkvng: Thinking of you:💕💘💕💗💞💞💘
💕💞💝💕💘💞💝💖💖💞💗💓

Talking with you:💗💕💕💓💞💘
💝💖💞💓💕💗💖💞💗💘💓💞

Me loving you:💕💖💗💓💞💞💖
💘💕💖💗💞💝💝💞💓💖💖💕

Your existe…
(<pymongo.results.UpdateResult object at 0x000001BE0A488E08>, 2)
RT @lgbtexid: Some people have never watched exid singing hot pink perfectly while drunk and it shows https://t.co/zCDKn1Aa7m
(<pymongo.results.UpdateResult object at 0x000001BE0A4951C8>, 1)
Justin Tucker just missed a kick in the postseason....😳
(<pymongo.results.UpdateResult object at 0x000001BE0A4880C8>, -2)
RT @IHLaking: so this is pretty much the coolest thing ever 😱 https://t.co/3VVp9exi1n
(<pymongo.results.UpdateResult object at 0x000001BE0A48D088>, 1)
RT @kingxkamm: Me blocking all the people trying to defend R.Kelly
#SurvivingRKelly #Lifetime #RKelly https://t.co/0lSrxD5e4J
(<pymongo.results.UpdateResult object at 0x000001BE0A488188>, -1)
RT @ayogukingsley_: My painting progress!!
         
2014             Vs 

...

(<pymongo.results.UpdateResult object at 0x000001BE0DF83788>, 0)
RT @marcellehteit: it sucks https://t.co/3kUKNFEU6J
(<pymongo.results.UpdateResult object at 0x000001BE0A4950C8>, -3)
RT @cloutboishorty: steps to a healthy relationship:

-be honest with her
-respect her space
-make her feel special 
-don’t be petty, grow…
(<pymongo.results.UpdateResult object at 0x000001BE0A488CC8>, 4)
Everyone be talking about that peach pie. It better be nice
(<pymongo.results.UpdateResult object at 0x000001BE0A488388>, 5)
I really really REALLY don’t like this https://t.co/1PuDtVfE9q
(<pymongo.results.UpdateResult object at 0x000001BE0A4882C8>, 2)
RT @its__maaarrrr: I figured https://t.co/koroIneF07
(<pymongo.results.UpdateResult object at 0x000001BE0A4952C8>, 0)
RT @realDonaldTrump: Excited to see our friends in Egypt opening the biggest Cathedral in the Middle East. President El-Sisi is moving his…
(<pymongo.results.UpdateResult object at 0x000001BE0A4950C8>, 3)
RT @C0REZI: harry potter, age 11: wh

(<pymongo.results.UpdateResult object at 0x000001BE0A495D88>, 1)
@MotherOfDoggons @USMC If a 50 year old guy waits around for an underage girl to get in his inbox, he's going to be waiting forever.
(<pymongo.results.UpdateResult object at 0x000001BE0A488048>, 0)
RT @EthicsAesthetic: Annihilation is on Hulu everyone, just FYI. https://t.co/ltzOiiWZRE
(<pymongo.results.UpdateResult object at 0x000001BE0A4880C8>, 0)
@FrazzleMyGimp @itsAmeAnthony Continuity
(<pymongo.results.UpdateResult object at 0x000001BE0A488408>, 0)
The latest Daily Foot! https://t.co/iDYOsJAo0x Thanks to @Yuki_Ogimi #長友佑都 #森保一
(<pymongo.results.UpdateResult object at 0x000001BE0A488D48>, 2)
RT @billyeichner: New Oscar host idea: @SnoopDogg.
(<pymongo.results.UpdateResult object at 0x000001BE0A488948>, 0)
RT @JohnStossel: The next step. https://t.co/URH3pA0l0a
(<pymongo.results.UpdateResult object at 0x000001BE0A488B48>, 0)
RT @MitchSchwartz72: Yes the 2013 playoff game when Pat was in high school affects our upcoming

(<pymongo.results.UpdateResult object at 0x000001BE0A488588>, 0)
And these are the type of people that the left would have run this country? Behind sad. And Twitter has no problem… https://t.co/klKh4iufZD
(<pymongo.results.UpdateResult object at 0x000001BE0A488908>, -1)
RT @JapanFocus: NEW: “It Ain’t Over ‘Till It’s Over”: Reflections on the Okinawan Anti-Base Resistance

C. Douglas Lummis

 https://t.co/aH…
(<pymongo.results.UpdateResult object at 0x000001BE0A4883C8>, 0)
RT @APCentralRegion: One suspect was white; the other was black. One had a gun; the other was unarmed. The two Milwaukee police encounters…
(<pymongo.results.UpdateResult object at 0x000001BE0A488608>, -1)
RT @kennwadiogbu: My pencil drawing progress

   2013           vs          2017 https://t.co/aSUbH5vZWs
(<pymongo.results.UpdateResult object at 0x000001BE0A488208>, 0)
@MemTigerFreak Gettting really tired of the constant coaching carousel in college football.
(<pymongo.results.UpdateResult object at 0x000001BE0A48

(<pymongo.results.UpdateResult object at 0x000001BE0A488B08>, 0)
@faith_crist Trying to piss off someone else’s shit off the pan and not doing it
(<pymongo.results.UpdateResult object at 0x000001BE0A488608>, -8)
RT @Crypto_Peanut: @oweasel1337 @winternomics @Derella @Derella Please DM @winternomics . A group of people are mass reporting his account…
(<pymongo.results.UpdateResult object at 0x000001BE0A471C48>, 1)
When ppl slander scorpios with me &gt;&gt;&gt;&gt;&gt;
(<pymongo.results.UpdateResult object at 0x000001BE0A4881C8>, 0)
RT @DVATW: @Biafran16Son @UKLabour Thank you. I note that they refuse to deal with the question I raise. Speaks volumes.
(<pymongo.results.UpdateResult object at 0x000001BE0A495D08>, 0)
RT @AnjaliEwing: @BrexitCentral @theresa_may your deal denies democracy by extending Brexit into the future with no guaranteed date for any…
(<pymongo.results.UpdateResult object at 0x000001BE0A4717C8>, -3)
RT @AviMayer: My Muslim Arab taxi driver, whose prayer rug is neatly f

(<pymongo.results.UpdateResult object at 0x000001BE0A488988>, 0)
You write RIP, someone committed suicide, why? He was depressed!
(<pymongo.results.UpdateResult object at 0x000001BE0A488CC8>, 1)
RT @princessharumi_: vamp slayer #AdventureTime #marceline #art https://t.co/YozxgQg2Mz
(<pymongo.results.UpdateResult object at 0x000001BE0A488AC8>, 0)
RT @untoIdtaegi: TAEHYUNG IS WEARING THE SHOOKY VERSION OF THE PUFF JACKET..... TAEGI NATION https://t.co/wbzxMRBso1
(<pymongo.results.UpdateResult object at 0x000001BE0A488388>, 0)
RT @TheCobrasConk: Human 10 pin bowling; from my mate’s old rugby club in Long Eaton https://t.co/752uFg0T0B
(<pymongo.results.UpdateResult object at 0x000001BE0A4883C8>, 0)
@psychedelizm de drag race
(<pymongo.results.UpdateResult object at 0x000001BE0A488108>, -1)
RT @NineTaiIsChakra: The Final Battle 🔥 https://t.co/Jc0nTCBMB4
(<pymongo.results.UpdateResult object at 0x000001BE0A488AC8>, -1)
You women be in ya snaps and it’s bright.
(<pymongo.results.UpdateResult 

In [77]:
#Verifying if the scores were well inserted into the documents
clean.find_one()

{'_id': ObjectId('5c5daf1a9d98244eefaee288'),
 'contributors': None,
 'coordinates': None,
 'created_at': 'Sun Jan 06 20:08:07 +0000 2019',
 'display_text_range': [12, 66],
 'entities': {'hashtags': [],
  'symbols': [],
  'urls': [],
  'user_mentions': [{'id': 1561123663,
    'id_str': '1561123663',
    'indices': [0, 11],
    'name': 'B/R Football',
    'screen_name': 'brfootball'}]},
 'favorite_count': 0,
 'favorited': False,
 'filter_level': 'low',
 'geo': None,
 'id': 1082005915723804672,
 'id_str': '1082005915723804672',
 'in_reply_to_screen_name': 'brfootball',
 'in_reply_to_status_id': 1082005301514199040,
 'in_reply_to_status_id_str': '1082005301514199040',
 'in_reply_to_user_id': 1561123663,
 'in_reply_to_user_id_str': '1561123663',
 'is_quote_status': False,
 'lang': 'en',
 'place': None,
 'quote_count': 0,
 'reply_count': 0,
 'retweet_count': 0,
 'retweeted': False,
 'score': -1,
 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>

In [162]:
#Creating index
clean.create_index([("score",pymongo.TEXT)], name="search_score", default_language="english")

'search_score'

In [163]:
#Counting the total documents
clean.count()

C:\Users\Ivan_\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  """Entry point for launching an IPython kernel.


2837

In [164]:
#Counting how many tweets there are score bigger than 0
clean.find( {"score": { "$gt": 0 }} ).count()

C:\Users\Ivan_\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  """Entry point for launching an IPython kernel.


837

In [165]:
#Counting how many tweets there are score less than 0
clean.find( {"score": { "$lt": 0 }} ).count()

C:\Users\Ivan_\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  """Entry point for launching an IPython kernel.


561

In [166]:
#Counting how many tweets there are score = 0
clean.find( {"score": 0} ).count()

C:\Users\Ivan_\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  """Entry point for launching an IPython kernel.


1324

In [167]:
#The tweet with the smallest score
list(clean.find({},{"_id":1,"score": 1,"text":1}).limit(1).skip(115).sort("score",1))

[{'_id': ObjectId('5c5daf1b9d98244eefaef56d'),
  'score': -13,
  'text': 'RT @shaylabruh: I’ll be damned if I raised a sorry ass nigga for a son Fr. I’ll slap the shit outta him'}]

#### OBS: As said before some documents there are more fields "Text" so when we stract the scores, it evaluetes the first field. we had a total of 115 documents empty in the first field, so we skipped those documents and got the value for the first valid document.

In [168]:
#The tweet with the biggest score
list(clean.find({},{"_id":1,"score": 1,"text":1}).limit(1).sort("score",-1))

[{'_id': ObjectId('5c5daf1c9d98244eefaf0174'),
  'score': 20,
  'text': 'RT @jacksonpbn: As it stands, APC can’t win Katsina, they can’t win Kano, they can’t win Kaduna, they can’t win Sokoto, they can’t win Jiga…'}]

In [169]:
#The 10 biggest scores in our collection
list(clean.find({},{"_id":1,"score": 1,"text":1}).limit(10).sort("score",-1))

[{'_id': ObjectId('5c5daf1c9d98244eefaf0174'),
  'score': 20,
  'text': 'RT @jacksonpbn: As it stands, APC can’t win Katsina, they can’t win Kano, they can’t win Kaduna, they can’t win Sokoto, they can’t win Jiga…'},
 {'_id': ObjectId('5c5daf1b9d98244eefaef79b'),
  'score': 11,
  'text': 'RT @lrocket: @mekkaokereke @laparisa Wow thanks Mekka!  Great to get some props. I am extremely proud of what we have achieved at @SpaceX a…'},
 {'_id': ObjectId('5c5daf1b9d98244eefaeef27'),
  'score': 10,
  'text': 'Our 2014 Gold medal winning Pinot Noir is a beautiful ruby colour and has a pronounced fruity aroma of cherries 🍒 w… https://t.co/lZSBJbLmxP'},
 {'_id': ObjectId('5c5daf1b9d98244eefaef198'),
  'score': 10,
  'text': 'RT @snowberrytae: the fond look in Namjoon’s eyes and his sweet smile everytime he encourage Taehyung to give a speech, he looks so proud a…'},
 {'_id': ObjectId('5c5daf1c9d98244eefaefa81'),
  'score': 10,
  'text': 'Hop along the 3,2,1 Giveaway Hop to win some awesome prizes

In [171]:
#The 10 smallest scores in our collection
list(clean.find({},{"_id":1,"score": 1,"text":1}).limit(10).skip(115).sort("score",1))

[{'_id': ObjectId('5c5daf1b9d98244eefaef56d'),
  'score': -13,
  'text': 'RT @shaylabruh: I’ll be damned if I raised a sorry ass nigga for a son Fr. I’ll slap the shit outta him'},
 {'_id': ObjectId('5c5daf1a9d98244eefaee4be'),
  'score': -11,
  'text': 'RT @Ultra_Suristic: I’m now (finally selfish enough to admit being) so pissed off at Zidane for leaving. We lost our bite with Cristiano bu…'},
 {'_id': ObjectId('5c5daf1b9d98244eefaeeb2a'),
  'score': -11,
  'text': 'Just finished watching I, Tonya...\n\nIt’s forever fuck these niggas bc they really will ruin their life. Wow.'},
 {'_id': ObjectId('5c5daf1b9d98244eefaef0c6'),
  'score': -9,
  'text': 'RT @marklutchman: It makes me so mad that when a cop kills someone, the entire city is out rioting.\n\nBut when a drug dealer kills a 7 year…'},
 {'_id': ObjectId('5c5daf1b9d98244eefaef2dd'),
  'score': -9,
  'text': 'Your tired of messing with these lame niggas have you ever had a nigga sing to ya?😙 #GirlsNeedLove \U0001f9e1😍… https://t.

In [176]:
#Exporting collection raw_tweets as dump
os.system("mongodump --db twitter --collection raw_tweets --out C:/Users/Ivan_/_Jupyter/Mongo ")

0

In [177]:
#Exporting collection clean_tweets as dump
os.system("mongodump --db twitter --collection clean_tweets --out C:/Users/Ivan_/_Jupyter/Mongo ")

0